In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
## creating transform
transformer_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                  transforms.RandomHorizontalFlip(p=0.5),
                                  transforms.RandomAffine(0, translate=(0.1,0.1)),
                                  transforms.ToTensor()
                                 ])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root=".",
                                             train=True,
                                             transform=transformer_train,
                                             download=False
                                            )

test_dataset = torchvision.datasets.CIFAR10(root=".",
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=False
                                           )

In [4]:
## creating loaders for the data
batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           shuffle=True,
                                           batch_size=batch_size
                                          )

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           shuffle=False,
                                           batch_size=batch_size
                                          )

In [5]:
## creating the model
class CNN(nn.Module):
    def __init__(self,K):
        super(CNN, self).__init__()

        ## using the same concept from VGG here
        self.conv1 = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, padding=1),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(32),
                                   nn.Conv2d(32, 32, kernel_size=3, padding=1),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(32),
                                   nn.MaxPool2d(2),
                                  )
        
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, padding=1),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64),
                                   nn.Conv2d(64, 64, kernel_size=3, padding=1),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64),
                                   nn.MaxPool2d(2),
                                  )
        
        self.conv3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, padding=1),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(128),
                                   nn.Conv2d(128, 128, kernel_size=3, padding=1),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(128),
                                   nn.MaxPool2d(2),
                                  )
    
        self.fc1 = nn.Linear(128*4*4,1024)
        self.fc2 = nn.Linear(1024, K)

    def forward(self, X):
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.conv3(X)
        X = X.view(X.size(0), -1)
        X = F.dropout(X, p=0.5)
        X = F.relu(self.fc1(X))
        X = F.dropout(X, p=0.2)
        X = self.fc2(X)

        return X

In [6]:
## creating training tools
labels = train_dataset.classes
model = CNN(len(labels))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cuda', index=0)

In [9]:
model.to(device)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel

In [10]:
## creating training function
def train_model(model, criterion, optimizer, train_loader, test_loader, epochs):
    train_losses = []
    test_losses = []
    
    for it in range(epochs):
        epoch_start_time = datetime.now()
        epoch_train_losses = []
        epoch_test_losses = []
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_train_losses.append(loss.item())
            
        train_losses.append(np.mean(epoch_train_losses))
        
        model.eval()
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            epoch_test_losses.append(loss.item())

        test_losses.append(np.mean(epoch_test_losses))
    
        dt = datetime.now() - epoch_start_time
    
        print(f"Epoch {it+1}/{epochs}, Train Loss: {train_losses[it]:.4f}, Test Loss: {test_losses[it]:.4f}, Duration: {dt}")
        
    return train_losses, test_losses

In [11]:
train_losses, test_losses = train_model(model, criterion, optimizer, train_loader, test_loader, 80)

Epoch 1/80, Train Loss: 1.4266, Test Loss: 1.1743, Duration: 0:00:22.042443
Epoch 2/80, Train Loss: 1.0172, Test Loss: 0.8871, Duration: 0:00:21.148833
Epoch 3/80, Train Loss: 0.8575, Test Loss: 0.7923, Duration: 0:00:21.077379
Epoch 4/80, Train Loss: 0.7580, Test Loss: 0.7175, Duration: 0:00:21.347542
Epoch 5/80, Train Loss: 0.6886, Test Loss: 0.6435, Duration: 0:00:21.077327
Epoch 6/80, Train Loss: 0.6511, Test Loss: 0.6674, Duration: 0:00:21.355493
Epoch 7/80, Train Loss: 0.6138, Test Loss: 0.6249, Duration: 0:00:21.245741
Epoch 8/80, Train Loss: 0.5800, Test Loss: 0.6971, Duration: 0:00:21.518240
Epoch 9/80, Train Loss: 0.5504, Test Loss: 0.5755, Duration: 0:00:21.097441
Epoch 10/80, Train Loss: 0.5271, Test Loss: 0.5884, Duration: 0:00:21.050143
Epoch 11/80, Train Loss: 0.5108, Test Loss: 0.5703, Duration: 0:00:21.040588
Epoch 12/80, Train Loss: 0.4914, Test Loss: 0.5379, Duration: 0:00:21.020975
Epoch 13/80, Train Loss: 0.4733, Test Loss: 0.5795, Duration: 0:00:21.150501
Epoch 14

In [15]:
## Getting accuracies
model.eval()
train_acc = 0
test_acc = 0
test_preds = np.array([])

## Train accuracy
n_corrects = 0
total = 0
for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    _, preds = torch.max(model(inputs), 1)
    n_corrects += (preds == targets).sum()
    total += targets.shape[0]

train_acc = n_corrects/total


## Test accuracy
n_corrects = 0
total = 0
for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    _, preds = torch.max(model(inputs), 1)
    test_preds = np.concatenate([test_preds, preds.cpu()])
    n_corrects += (preds == targets).sum()
    total += targets.shape[0]

test_acc = n_corrects/total

print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")

Train Accuracy: 0.9411999583244324, Test Accuracy: 0.8837999701499939


In [16]:
X_test = test_dataset.data
y_test = test_dataset.targets

In [17]:
fig = plt.figure(figsize=(8,8))
cm = confusion_matrix(y_test, test_preds)
sns.heatmap(cm, annot=True, fmt="", square=True, cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.show()

NameError: name 'sns' is not defined

<Figure size 800x800 with 0 Axes>

In [ ]:
misclassifications = np.where(test_preds != y_test)[0]
mis_idx = np.random.choice(misclassifications)
plt.imshow(X_test[mis_idx])
plt.title(f"True Label {labels[y_test[mis_idx]]} Predicted Label {labels[int(test_preds[mis_idx])]}")
plt.show()

In [52]:
from torchsummary import summary

In [53]:
summary(model, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
            Conv2d-4           [-1, 32, 32, 32]           9,248
              ReLU-5           [-1, 32, 32, 32]               0
       BatchNorm2d-6           [-1, 32, 32, 32]              64
         MaxPool2d-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          18,496
              ReLU-9           [-1, 64, 16, 16]               0
      BatchNorm2d-10           [-1, 64, 16, 16]             128
           Conv2d-11           [-1, 64, 16, 16]          36,928
             ReLU-12           [-1, 64, 16, 16]               0
      BatchNorm2d-13           [-1, 64, 16, 16]             128
        MaxPool2d-14             [-1, 6